# Problem Set 2 - Part 3 (VG): Ambiguity and Inference

This lab should be completed individually.

It extend the topics in the previous two parts with additional questions.

This assignment has 27 marks in total.

### Pre-requisite knowledge

From `problem-set-1`:
- First order logic
- Lambda calculus
- Feature unification context free grammar

From `problem-set-2-part1.ipynb`:
- Cooper storage

From `problem-set-2-part2.ipynb`:
- Using Prover9
- Model building with `mace4`


In [1]:
# This task needs NLTK and Jupyter Notebook (IPython package).

import nltk
from nltk.grammar import FeatureGrammar
from nltk.sem import cooper_storage as cs

from utils import display_latex, display_translation, display_tree, display, Markdown
from itertools import zip_longest

read_expr = nltk.sem.Expression.fromstring

from utils2 import sem_parser, evaluate_sentences, syntax, syntax_notv, compare_synsem, syntax_storage

import re

## Types in Cooper Storage [12 marks in total]

### 1. Types of syntactic categories [5 marks in total]

In this question we are going to look at alternative representations for transitive verbs and verb phrases from what you saw in `problem-set-2-part1`. 
If you look carefully you see that the `TV`, `DTV` categories have simpler representations.
That means that `VP`s have to have more complex representations as shown below.
If you run the code below you will see that it produces the same readings that you saw earlier with a different grammar.

In [2]:
fcfg_string = r"""
VP[NUM=?n,SEM=<\x.(?obj(?v(x)))>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<\x.(?pp(\y.?obj(?v(x, y))))>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]
VP[NUM=?n,SEM=<\x.(?obj(\z.?pp(\y.?v(x, y, z))))>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]

TV[NUM=sg,SEM=<\x y.bite(x,y)>,TNS=pres] -> 'bites'
TV[NUM=pl,SEM=<\x y.bite(x,y)>,TNS=pres] -> 'bite'
DTV[NUM=sg,SEM=<\x z y.give(x,y,z)>,TNS=pres] -> 'gives'
DTV[NUM=pl,SEM=<\x z y.give(x,y,z)>,TNS=pres] -> 'give'
"""

# this is going to add new rules to the syntax:
new_syntax = FeatureGrammar(
    syntax_notv.start(),
    syntax_notv.productions() + FeatureGrammar.fromstring(fcfg_string).productions()
)

sentences = [
    'all dogs bite a bone',
    'a man gives a bone to every dog',
    'Russia gives Moscow to Napoleon',
]

# change `verbose` to see the trees
sents_reps = sem_parser(sentences, new_syntax, verbose=False)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    for semrep in semreps:
        display_translation(f"{i}. {sent}", semrep)

"0. all dogs bite a bone": $\forall\ x.(dog(x)\ \to\ \exists\ z_{1}.(bone(z_{1})\ \land\ bite(x,z_{1})))$

"1. a man gives a bone to every dog": $\exists\ x.(man(x)\ \land\ \exists\ z_{7}.(bone(z_{7})\ \land\ \forall\ z_{6}.(dog(z_{6})\ \to\ give(x,z_{7},z_{6}))))$

"1. a man gives a bone to every dog": $\exists\ x.(man(x)\ \land\ \forall\ z_{4}.(dog(z_{4})\ \to\ \exists\ z_{3}.(bone(z_{3})\ \land\ give(x,z_{3},z_{4}))))$

"2. Russia gives Moscow to Napoleon": $give(russia,moscow,napoleon)$

The aim of this question is to inspect and compare two sets of grammar rules from `syntax`, `new_syntax`:

In [3]:
# display the comparison
table = """
| Category | Grammar 1 | Grammar 2 
|----------|-----------|----------
"""
for cat, sem1, sem2 in compare_synsem(syntax, new_syntax):
    if cat[0] in ['VP', 'DTV', 'TV', 'IV','NP', 'S', 'N']:
        table += f"| {cat[0]} -> {cat[1]} | {'-' if sem1 is None else sem1} | {'-' if sem2 is None else sem2}\n"

display(Markdown(table))


| Category | Grammar 1 | Grammar 2 
|----------|-----------|----------
| DTV -> {{STR}} | \Y X x.X(\z.Y(\y.give(x,y,z))) | \x z y.give(x,y,z)
| N -> {{STR}} | \x.bone(x) | \x.bone(x)
| N -> {{STR}} | \x.dog(x) | \x.dog(x)
| N -> {{STR}} | \x.man(x) | \x.man(x)
| NP -> Det Nom | ?det(?nom) | ?det(?nom)
| NP -> PropN | ?np | ?np
| S -> NP VP | ?subj(?vp) | ?subj(?vp)
| TV -> {{STR}} | \X x.X(\y.bite(x,y)) | \x y.bite(x,y)
| VP -> DTV NP PP | ?v(?obj,?pp) | \x.?pp(\y.?obj(?v(x,y)))
| VP -> DTV NP PP | - | \x.?obj(\z.?pp(\y.?v(x,y,z)))
| VP -> IV | ?v | ?v
| VP -> TV NP | ?v(?obj) | \x.?obj(?v(x))


**1a.** What are the semantic types of each syntactic category?
Compare `new_syntax` with `syntax` imported form `util2.py`. 

Write types in `???`. **[4 marks]**

|	Syntactic Type	|	Grammar 1			        |	Grammar 2			      | 
|-------------------|:------------------------------|:----------------------------| 
| S					| `t`					        | `t`					      |
| DET				| `<<e,t>,<<e,t>,t>>`	        | `<<e,t>,e>`	      |
| N					| `<e,t>`				        | `<e,t>`				      |
| IV				| `<e,t>`					    | `<e,t>`				      |
| TV				| `<<<e,t>,t>,<e,t>>`		    | `<e,<e,t>>`				  |
| DTV				| `<<<e,t>,t>,<<<e,t>,t>,<e,t>>>`| `<e,<e,<e,t>>>`			  |
| NP				| `<<e,t>,t>`					| `<e>`			  |
| PP				| `<<e,t>,t>`					| `	<e>`			      |
| VP				| `<e,t>`					    | `<e,t>`					  |


**1b.** There are two simple ways to construct lambda terms in lambda calculus:

|	Lambda Syntax	|	Name				|	Description			|
|-------------------|-----------------------|-----------------------|
|	P(X)			|	application (app)	|	applying P on X.	|
|	\x.P			|	abstraction (abs)	|	function definition.|

Compare the composition of non-terminal nodes as implemented in two grammars **[1 mark]**:

|	Syntactic Type		|	Grammar 1		|	Grammar 2		| 
|-----------------------|-------------------|-------------------| 
|	`S  -> NP VP`		|	app				|	app				|
|	`VP -> IV`			|	none			|	none			|
|	`NP -> DET Nom`		|	`abs`		    |	`app`	        |
|	`VP -> TV NP`		|	`app`	        |	abs,app			|
|	`VP -> DTV NP PP`	|	`app, app`			|	`abs, app, app`		|


### 2. A new conjunction rule [7 marks in total]

The grammar below implements Cooper Storage - in fact this version is included as `storage.fcfg` with NLTK. In the lecture and in the previous part of this assignment we used our modified version. The goal of this question is to examine why we had to modify this version of the grammar. In this grammar the lexical representations of verbs are basic but their compositions with NPs are not complex as in Question 1 of this notebook. Therefore, this grammar also requires a different conjunction rule from the one in `problem-set-2-part1`.

Run the code below and answer questions:

In [4]:
fcfg_storage = r"""
%start S

S[SEM=[CORE=<?vp(?subj)>, STORE=(?b1+?b2)]] -> NP[SEM=[CORE=?subj, STORE=?b1]] VP[SEM=[CORE=?vp, STORE=?b2]]

VP[SEM=?v] -> IV[SEM=?v]
VP[SEM=[CORE=<?v(?obj)>, STORE=(?b1+?b2)]] -> TV[SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]]
VP[SEM=[CORE=<?v(?pp)(?obj)>, STORE=(?b1+?b2+?b3)]] -> DTV[SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]] PP[+TO, SEM=[CORE=?pp, STORE=?b3]]  

NP[SEM=[CORE=<@x>, STORE=((<bo(?det(?nom),@x)>)+?b1+?b2)]] -> Det[SEM=[CORE=?det, STORE=?b1]] Nom[SEM=[CORE=?nom, STORE=?b2]]
Nom[SEM=?n] -> N[SEM=?n]
NP[SEM=?np] -> PropN[SEM=?np]

PP[+TO, SEM=[CORE=?np, STORE=?b1]] -> P NP[SEM=[CORE=?np, STORE=?b1]]

# Lexical items:
Det[SEM=[CORE=<\Q P.exists x.(Q(x) & P(x))>, STORE=(/)]] -> 'a'
Det[SEM=[CORE=<\Q P.all x.(Q(x) implies P(x))>, STORE=(/)]] -> 'every'

N[SEM=[CORE=<\x.library(x)>, STORE=(/)]] -> 'library' 
N[SEM=[CORE=<\x.book(x)>, STORE=(/)]] -> 'book' 
N[SEM=[CORE=<\x.girl(x)>, STORE=(/)]] -> 'girl'
N[SEM=[CORE=<\x.boy(x)>, STORE=(/)]] -> 'boy'

IV[SEM=[CORE=<\x.smile(x)>, STORE=(/)]] -> 'smiles' 
IV[SEM=[CORE=<\x.walk(x)>, STORE=(/)]] -> 'walks'

TV[SEM=[CORE=<\y x.read(x,y)>, STORE=(/)]] -> 'reads' 

DTV[SEM=[CORE=<\z y x.give(x,y,z)>, STORE=(/)]] -> 'gives' 

PropN[SEM=[CORE=<@x>, STORE=(<bo(\P.P(angus),@x)>)]] -> 'Angus' 
PropN[SEM=[CORE=<@x>, STORE=(<bo(\P.P(cyril),@x)>)]] -> 'Cyril'

P[+TO] -> 'to'
"""

Let us compare the construction of representations in `SEM.CORE` with the grammar in Question 1:

In [5]:
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)

# display the comparison
table = """
| Category | Grammar 2 | Grammar 3 - CORE | Grammar 3 - STORE
|----------|-----------|------------------|-------------------
"""
for cat, sem1, sem2 in compare_synsem(cs_syntax, new_syntax):
    if cat[0] in ['VP', 'DTV', 'TV', 'NP', 'S', 'PP']:
        if sem1 is not None and type(sem1)!=nltk.grammar.FeatStructNonterminal:
             # then there is a semantic representation, but its not structured:
            core = sem1
            store = '-'
        elif type(sem1)==nltk.grammar.FeatStructNonterminal:
            # then the semantic representation is structured:
            core = str(sem1['CORE'])
            store = ""
            # find the binding operations introduced here:
            for bo in re.findall(r'\(bo\((.+)\)\)', str(sem1['STORE'])):
                rep, ref = tuple(bo.split(','))
                store += f"{ref} --> {rep}" 
                
            # if there is no new binding then ignore the store:
            if store == "":
                store = '-'
        else:
            # then there is no semantic representation:
            core = '-'
            store = '-'
            
        if sem2 is None:
            sem2 = '-'
            
        table += f"| `{cat[0]} -> {cat[1]}` " +\
                 f"| `{sem2}` " +\
                 f"| `{core}` " +\
                 f"| `{store}` " +\
                  "\n"

display(Markdown(table))


| Category | Grammar 2 | Grammar 3 - CORE | Grammar 3 - STORE
|----------|-----------|------------------|-------------------
| `DTV -> {{STR}}` | `\x z y.give(x,y,z)` | `-` | `-` 
| `DTV -> {{STR}}` | `\x z y.give(x,y,z)` | `\z y x.give(x,y,z)` | `-` 
| `NP -> Det Nom` | `?det(?nom)` | `@x` | `@x --> ?det(?nom)` 
| `NP -> PropN` | `?np` | `?np` | `-` 
| `PP -> P NP` | `?np` | `?np` | `-` 
| `S -> NP VP` | `?subj(?vp)` | `?vp(?subj)` | `-` 
| `TV -> {{STR}}` | `\x y.bite(x,y)` | `\y x.read(x,y)` | `-` 
| `TV -> {{STR}}` | `\x y.bite(x,y)` | `-` | `-` 
| `VP -> DTV NP PP` | `\x.?pp(\y.?obj(?v(x,y)))` | `?v(?pp,?obj)` | `-` 
| `VP -> DTV NP PP` | `\x.?obj(\z.?pp(\y.?v(x,y,z)))` | `-` | `-` 
| `VP -> IV` | `?v` | `?v` | `-` 
| `VP -> TV NP` | `\x.?obj(?v(x))` | `?v(?obj)` | `-` 


**2a.** What are the types of representations in `SEM.CORE` for each syntactic category? **[2 marks]**

|	Syntactic Type	|	Grammar 3 -	CORE	
|-------------------|-----------------------
| S					| `t`					
| Det				| `<<e,t>,e>`					
| N					| `<e,t>`					
| IV				| `<e,t>`					
| TV				| `<e,<e,t>>`					
| DTV				| `<e,<e,<e,t>>>`					
| NP				| `<e>`					
| PP				| `<e>`					
| VP				| `<e,t>`					


**2b.** Compare the composition of non-terminal nodes in Cooper Storage and the grammar from the previous question. Fill in `???`. **[1 mark]**

|	Syntactic Type		|	Grammar Basic-CS	|	Grammar 2		| 
|-----------------------|-----------------------|-------------------| 
|	`S  -> NP VP`		|	`app`				|	app				|
|	`VP -> IV`			|	none				|	none			|
|	`NP -> DET Nom`		|	`store: app`				|	`app`			|
|	`VP -> TV NP`		|	`app`				|	abs/app			|
|	`VP -> DTV NP PP`	|	`app`				|	`abs/app/app`			|


**2c.** What are other differences between two representations?. **[1 mark]**

**2d.** Finish the rule where the conjunction is handled in the  `STORE` by replacing `xxx` with an appropriate lambda function and then uncomment the rule. **[2 marks]**

In [6]:
fcfg_conj = r"""
# Conjunction rule:
CONJ -> 'and'
NP[SEM=[CORE=<@x>, STORE=((<bo(\Q.(Q(?n1) & Q(?n2)), @x)>)+?b1+?b2)]] -> NP[SEM=[CORE=?n1, STORE=?b1]] CONJ NP[SEM=[CORE=?n2, STORE=?b2]]
#NP[SEM=[CORE=<yyy>, STORE=(?b1+?b2)]] -> NP[SEM=[CORE=?n1, STORE=?b1]] CONJ NP[SEM=[CORE=?n2, STORE=?b2]]
"""

cs_syntax = FeatureGrammar.fromstring(fcfg_storage+fcfg_conj)

sentences = [
    'every library gives a book to a girl and a boy',
]

sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

counter = 0
for i, (sent, semreps) in enumerate(sents_reps.items()):
    for semrep in semreps:
        # check if it is valid
        if str(semrep).find("@") == -1:
            counter+=1
            display_translation(f"{counter}", semrep)

"1": $\exists\ x.(boy(x)\ \land\ \exists\ z_{13}.(girl(z_{13})\ \land\ \exists\ z_{11}.(book(z_{11})\ \land\ \forall\ z_{10}.(library(z_{10})\ \to\ give(z_{10},z_{11},z_{13})))\ \land\ \exists\ z_{12}.(book(z_{12})\ \land\ \forall\ z_{10}.(library(z_{10})\ \to\ give(z_{10},z_{12},x)))))$

"2": $\exists\ x.(girl(x)\ \land\ \exists\ z_{17}.(boy(z_{17})\ \land\ \exists\ z_{15}.(book(z_{15})\ \land\ \forall\ z_{14}.(library(z_{14})\ \to\ give(z_{14},z_{15},x)))\ \land\ \exists\ z_{16}.(book(z_{16})\ \land\ \forall\ z_{14}.(library(z_{14})\ \to\ give(z_{14},z_{16},z_{17})))))$

"3": $\exists\ x.(boy(x)\ \land\ \exists\ z_{35}.(girl(z_{35})\ \land\ \exists\ z_{34}.(book(z_{34})\ \land\ \forall\ z_{32}.(library(z_{32})\ \to\ give(z_{32},z_{34},z_{35}))\ \land\ \forall\ z_{33}.(library(z_{33})\ \to\ give(z_{33},z_{34},x)))))$

"4": $\exists\ x.(girl(x)\ \land\ \exists\ z_{39}.(boy(z_{39})\ \land\ \exists\ z_{38}.(book(z_{38})\ \land\ \forall\ z_{36}.(library(z_{36})\ \to\ give(z_{36},z_{38},x))\ \land\ \forall\ z_{37}.(library(z_{37})\ \to\ give(z_{37},z_{38},z_{39})))))$

"5": $\exists\ x.(boy(x)\ \land\ \exists\ z_{43}.(book(z_{43})\ \land\ \exists\ z_{42}.(girl(z_{42})\ \land\ \forall\ z_{40}.(library(z_{40})\ \to\ give(z_{40},z_{43},z_{42}))\ \land\ \forall\ z_{41}.(library(z_{41})\ \to\ give(z_{41},z_{43},x)))))$

...

"36": $\exists\ x.(book(x)\ \land\ \exists\ z_{249}.(girl(z_{249})\ \land\ \forall\ z_{248}.(library(z_{248})\ \to\ \exists\ z_{247}.(boy(z_{247})\ \land\ give(z_{248},x,z_{249})\ \land\ give(z_{248},x,z_{247})))))$

"37": $\exists\ x.(girl(x)\ \land\ \forall\ z_{252}.(library(z_{252})\ \to\ \exists\ z_{251}.(book(z_{251})\ \land\ \exists\ z_{250}.(boy(z_{250})\ \land\ give(z_{252},z_{251},x)\ \land\ give(z_{252},z_{251},z_{250})))))$

"38": $\forall\ x.(library(x)\ \to\ \exists\ z_{255}.(girl(z_{255})\ \land\ \exists\ z_{254}.(book(z_{254})\ \land\ \exists\ z_{253}.(boy(z_{253})\ \land\ give(x,z_{254},z_{255})\ \land\ give(x,z_{254},z_{253})))))$

"39": $\exists\ x.(book(x)\ \land\ \forall\ z_{258}.(library(z_{258})\ \to\ \exists\ z_{257}.(girl(z_{257})\ \land\ \exists\ z_{256}.(boy(z_{256})\ \land\ give(z_{258},x,z_{257})\ \land\ give(z_{258},x,z_{256})))))$

"40": $\forall\ x.(library(x)\ \to\ \exists\ z_{261}.(book(z_{261})\ \land\ \exists\ z_{260}.(girl(z_{260})\ \land\ \exists\ z_{259}.(boy(z_{259})\ \land\ give(x,z_{261},z_{260})\ \land\ give(x,z_{261},z_{259})))))$

**2b.** Is it possible to write a conjunction rule in `yyy` in the `CORE`? Why? **[1 marks]**

## Inference [15 marks in total]

### 3. Ambiguous syntax and semantics and inference [15 marks in total]

Extend the grammar in `problem-set-2-part2` to cover two sentences below that are similar to FraCaS problem 024. 

- no delegate obtained interesting results from the survey
- no delegate obtained results from the survey

**3a.** Extend the grammar with a lexical entry for `obtained` as a `TV` category where it is combined with only one object. **[6 marks]**  
Hint: Add a rule that attaches the `PP` to a bare plural `NP` without a `DET`.

**3b.** Also add another rule for `obtained` where this is a `DTV` category. **[3 marks]**

In [7]:
# your answers
fcfg_storage_answers_a = r"""
## Cover lexicon:
Det[NUM=?n, SEM=[CORE=<\P.\Q. (- exists x.(P(x) & Q(x)))>, STORE=(/)]] -> 'no'
Det[NUM=?n, SEM=[CORE=<\P.\Q. exists x.((P(x) & Q(x)) & all y.(P(y) -> (x = y)))>, STORE=(/)]] -> 'the'

N[NUM=sg, SEM=[CORE=<\x.delegate(x)>, STORE=(/)]] -> 'delegate'
N[NUM=pl, SEM=[CORE=<\x.result(x)>, STORE=(/)]] -> 'results'
N[NUM=sg, SEM=[CORE=<\x.survey(x)>, STORE=(/)]] -> 'survey'
              
P1[SEM=[CORE=<\X.\P.\Q. exists x.(X(\y.(P(x) & Q(x) & from(x,y))))>, STORE=(/)]] -> 'from'
P2[SEM=[CORE=<\X.\Y.\Q. (Y(\x.(X(\y.(Q(x) & from(x,y))))))>, STORE=(/)]] -> 'from'

# Adj[SEM=[CORE=<\x.interesting(x)>, STORE=(/)]] -> 'interesting'
Adj[SEM=[CORE=<\P.\Q. exists x.(P(x) & interesting(x) & Q(x))>, STORE=(/)]] -> 'interesting'

PP1[SEM=[CORE=<?p(?np)>, STORE=(?b1+?b2)]] -> P1[SEM=[CORE=?p, STORE=?b1]] NP[Num=?n, SEM=[CORE=?np, STORE=?b2]]
PP2[SEM=[CORE=<?p(?np)>, STORE=(?b1+?b2)]] -> P2[SEM=[CORE=?p, STORE=?b1]] NP[Num=?n, SEM=[CORE=?np, STORE=?b2]]
# PP2[SEM=<?p(?det(?nom))>] -> P2[SEM=?p] Det[SEM=?det] Nom[SEM=?nom]

# add any missing rules
AP[SEM=[CORE=<?adj(?nom)>, STORE=(?b1+?b2)]] -> Adj[SEM=[CORE=?adj, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
# AP[SEM=<\Q.Q(x)(\P.(?adj(P) & ?nom(P)))>] -> Adj[SEM=?adj] Nom[SEM=?nom]
# NP[SEM=?n] -> AP[SEM=?n]

NP[NUM=?n, SEM=[CORE=<?pp(?nom)>, STORE=(?b1+?b2)]] -> Nom[NUM=?n, SEM=[CORE=?nom, STORE=?b1]] PP1[SEM=[CORE=?pp, STORE=?b2]]
NP[NUM=?n,SEM=[CORE=<?pp(?ap)>, STORE=(?b1+?b2)]] -> AP[NUM=?n,SEM=[CORE=?ap, STORE=?b1]] PP2[NUM=?n,SEM=[CORE=?pp, STORE=?b2]]

TV[SEM=[CORE=<\X.\x. X(\y.obtaine(x,y))>, STORE=(/)], TNS=past] -> 'obtained'
# VP[SEM=[CORE=<?v(?np,?pp)>, STORE=(?b1+?b2+?b3)]] -> DTV[NUM=?n, SEM=[CORE=?v, STORE=?b1]] PP3[NUM=?n, SEM=[CORE=?pp, STORE=?b2]] NP[SEM=[CORE=?np, STORE=?b3]] 
# VP[SEM=<?v(?obj,?pp)>] -> DTV[SEM=?v] NP2[SEM=?obj] PP2[SEM=?pp]

"""

fcfg_storage_answers_b = r"""
## Cover lexicon:
DTV[SEM=[CORE=<\Y.\X.\x. X(\z.Y(\y.obtaine(x,y,z)))>, STORE=(/)],TNS=past] -> 'obtained'
# add any missing rules
"""

sentences = [
    "no delegate obtained interesting results from the survey",
    "no delegate obtained results from the survey",
]

# this is going to add new rules to the syntax:
grammar = FeatureGrammar(
    syntax_storage.start(),
    syntax_storage.productions() + \
    FeatureGrammar.fromstring(fcfg_storage_answers_a).productions() + \
    FeatureGrammar.fromstring(fcfg_storage_answers_b).productions()
)

# change `verbose` to see the trees
sentence_readings = sem_parser(sentences, grammar, is_cs=True, verbose=False)

for i, (sent, semreps) in enumerate(sentence_readings.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)


1. no delegate obtained interesting results from the survey


"1": $\exists\ x.(survey(x)\ \land\ -\exists\ z_{444}.(delegate(z_{444})\ \land\ \exists\ z_{442}.(result(z_{442})\ \land\ interesting(z_{442})\ \land\ obtaine(z_{444},z_{442})\ \land\ from(z_{442},x)))\ \land\ \forall\ y.(survey(y)\ \to\ (x\ =\ y)))$

"2": $-\exists\ x.(delegate(x)\ \land\ \exists\ z_{445}.(survey(z_{445})\ \land\ \exists\ z_{442}.(result(z_{442})\ \land\ interesting(z_{442})\ \land\ obtaine(x,z_{442})\ \land\ from(z_{442},z_{445}))\ \land\ \forall\ y.(survey(y)\ \to\ (z_{445}\ =\ y))))$

2. no delegate obtained results from the survey


"1": $\exists\ x.(survey(x)\ \land\ -\exists\ z_{446}.(delegate(z_{446})\ \land\ \exists\ z_{443}.(result(z_{443})\ \land\ obtaine(z_{446},z_{443})\ \land\ from(z_{443},x)))\ \land\ \forall\ y.(survey(y)\ \to\ (x\ =\ y)))$

"2": $-\exists\ x.(delegate(x)\ \land\ \exists\ z_{447}.(survey(z_{447})\ \land\ \exists\ z_{443}.(result(z_{443})\ \land\ obtaine(x,z_{443})\ \land\ from(z_{443},z_{447}))\ \land\ \forall\ y.(survey(y)\ \to\ (z_{447}\ =\ y))))$

**3c.** Use Mace to show whether any of the readings of the first sentence entail any of the readings of the second sentence. **[1 mark]**

In [16]:
# Use apply_model_builder from problem-set-2-part2
def apply_model_builder(premises,goals):
    for premise in premises:
        for goal in goals:
            print("Premise : %s" % (premise))
            print("Goal    : %s" % (goal))
            mc1 = nltk.MaceCommand(goal, assumptions=[premise, a1, a2, a3])
            mc2 = nltk.MaceCommand(None, assumptions=[premise, goal, a1, a2, a3])
            mc1.build_model()
            mc2.build_model()
            print("a model-1:")
            print(mc1.valuation)
            print()
            print("a model-2:")
            print(mc2.valuation)
            print(10*'...')
            print()
# There is something missing in this code.
mypremises = sentence_readings['no delegate obtained results from the survey']
mygoals = sentence_readings['no delegate obtained interesting results from the survey']
a1 = read_expr('all x. (result(x) -> -(delegate(x) | survey(x)))')
a2 = read_expr('all x. (delegate(x) -> -(result(x) | survey(x)))')
a3 = read_expr('exist x. (result(x) & interesting(x))')
apply_model_builder(mypremises,mygoals)

Premise : exists x.(survey(x) & -exists z446.(delegate(z446) & exists z443.(result(z443) & obtaine(z446,z443) & from(z443,x))) & all y.(survey(y) -> (x = y)))
Goal    : exists x.(survey(x) & -exists z444.(delegate(z444) & exists z442.(result(z442) & interesting(z442) & obtaine(z444,z442) & from(z442,x))) & all y.(survey(y) -> (x = y)))
a model-1:
{}

a model-2:
{'C1': 'a',
 'C2': 'a',
 'C3': 'b',
 'delegate': set(),
 'from': set(),
 'interesting': {('b',)},
 'obtaine': set(),
 'result': {('b',)},
 'survey': {('a',)}}
..............................

Premise : exists x.(survey(x) & -exists z446.(delegate(z446) & exists z443.(result(z443) & obtaine(z446,z443) & from(z443,x))) & all y.(survey(y) -> (x = y)))
Goal    : -exists x.(delegate(x) & exists z445.(survey(z445) & exists z442.(result(z442) & interesting(z442) & obtaine(x,z442) & from(z442,z445)) & all y.(survey(y) -> (z445 = y))))
a model-1:
{}

a model-2:
{'C1': 'a',
 'C2': 'b',
 'delegate': set(),
 'from': set(),
 'interesting': {(

**3d.** Use Prover9 to show which readings of the second sentence entail a reading of the first sentence. **[1 mark]**

In [19]:
# Use apply_theorem_prover from problem-set-2-part2

# There is something missing in this code.
#apply_theorem_prover(...,...)
prover = nltk.Prover9()
# prover.config_prover9(r'/home/guszarzmo@GU.GU.SE/Programs/prover9/LADR-2009-11A/bin')

def apply_theorem_prover(premises, goals):
    for premise in premises:
        for goal in goals:
            print("Premise      : %s" % (premise))
            print("Goal         : %s" % (goal))
            print("Prover result: %s" % (prover.prove(goal, [premise])))
            print(10*'----')
mypremises = sentence_readings['no delegate obtained results from the survey']
mygoals = sentence_readings['no delegate obtained interesting results from the survey']
apply_theorem_prover(mypremises,mygoals)

Premise      : exists x.(survey(x) & -exists z446.(delegate(z446) & exists z443.(result(z443) & obtaine(z446,z443) & from(z443,x))) & all y.(survey(y) -> (x = y)))
Goal         : exists x.(survey(x) & -exists z444.(delegate(z444) & exists z442.(result(z442) & interesting(z442) & obtaine(z444,z442) & from(z442,x))) & all y.(survey(y) -> (x = y)))
Prover result: True
----------------------------------------
Premise      : exists x.(survey(x) & -exists z446.(delegate(z446) & exists z443.(result(z443) & obtaine(z446,z443) & from(z443,x))) & all y.(survey(y) -> (x = y)))
Goal         : -exists x.(delegate(x) & exists z445.(survey(z445) & exists z442.(result(z442) & interesting(z442) & obtaine(x,z442) & from(z442,z445)) & all y.(survey(y) -> (z445 = y))))
Prover result: True
----------------------------------------
Premise      : -exists x.(delegate(x) & exists z447.(survey(z447) & exists z443.(result(z443) & obtaine(x,z443) & from(z443,z447)) & all y.(survey(y) -> (z447 = y))))
Goal        

**3e.** Possibly there is a reading of the second sentence that does not entail the first sentence.
Which is it? 
Explain why the entailment does not hold? Does this reading agree with our intuitions about the meaning of the sentence? **[1 + 2 + 1 = 4 marks]**

Entailment does not hold in one reading. I would say this is because of the scope of negation. Negation transfer logical representation. In our case, this transformation affects the entailment of the two readings. 